In [ ]:
# Import necessary libraries
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
import face_recognition
import threading


# Get a reference to webcam #0 (the default one)
cap = cv2.VideoCapture(0)

# Load a sample picture and learn how to recognize it.
oliver_image = face_recognition.load_image_file("Oliver.jpg")
oliver_face_encoding = face_recognition.face_encodings(oliver_image)[0]


jesse_image = face_recognition.load_image_file("Jesse.jpg")
jesse_face_encoding = face_recognition.face_encodings(jesse_image)[0]



# Create arrays of known face encodings and their names
known_face_encodings = [
    oliver_face_encoding,
    jesse_face_encoding
]
known_face_names = [
    "Oliver Hein",
    "Jesse Grootjen"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []

# Initialize global variables
authorized_face_encoding = None  # Placeholder for the authorized user's face encoding
current_user_authorized = False  # Flag for authorization status


def check_user_authorization():
    global current_user_authorized
    global authorized_face_encoding
    
    while True:
        # Grab a single frame of video
        ret, frame = cap.read()
        
        if not ret:
            continue


        current_user_authorized = False
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                if(name=="Oliver Hein"):
                    current_user_authorized = True
            face_names.append(name)
        
        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        cv2.waitKey(10)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break




        

def paint_program():
    # Paint application logic
    def start_paint(event):
        if not current_user_authorized:
            messagebox.showerror("Access Denied", "Unauthorized user detected!")
            return
        canvas.old_coords = event.x, event.y

    def draw(event):
        if not current_user_authorized:
            return
        x, y = event.x, event.y
        old_x, old_y = getattr(canvas, 'old_coords', (None, None))
        if old_x and old_y:
            canvas.create_line(old_x, old_y, x, y, width=5, fill='black')
        canvas.old_coords = x, y

    def reset_coords(event):
        canvas.old_coords = None

    def clear_canvas(event):
        if not current_user_authorized:
            messagebox.showerror("Access Denied", "Unauthorized user detected!")
            return
        canvas.delete("all")

    # Create the GUI
    root = tk.Tk()
    root.title("Secure Paint Application")

    canvas = tk.Canvas(root, width=800, height=600, bg='white')
    canvas.pack()

    canvas.bind("<Button-1>", start_paint)        # Left click to start painting
    canvas.bind("<B1-Motion>", draw)             # Drag mouse while holding left button to draw
    canvas.bind("<ButtonRelease-1>", reset_coords)  # Release left click to stop drawing
    canvas.bind("<Button-3>", clear_canvas)      # Right click to clear canvas

    root.mainloop()



# Start a separate thread to handle paint program
paint_thread = threading.Thread(target=paint_program, daemon=True)
paint_thread.start()

# Launch the user authorization 
check_user_authorization()



cap.release()
cv2.destroyAllWindows()